# Branch and Bound algorithm for general system function

A system function needs to be coherent, i.e. (1) a high state of component events mean their better performance, and (2) a better (worse) component vector state does not lead to a worse (better) system state.

## System function

A system function takes a component vector state and information of component events as input.
It returns a system state, associated system value, and, if available, a minimum required state of component vector.

For example, consider a transport system 

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import copy

from BNS_JT import trans, variable, branch

HOME = os.getcwd()

# Network
node_coords = {'n1': (-2, 3),
               'n2': (-2, -3),
               'n3': (2, -2),
               'n4': (1, 1),
               'n5': (0, 0)}

arcs = {'e1': ['n1', 'n2'],
	'e2': ['n1', 'n5'],
	'e3': ['n2', 'n5'],
	'e4': ['n3', 'n4'],
	'e5': ['n3', 'n5'],
	'e6': ['n4', 'n5']}

arcs_avg_kmh = {'e1': 40,
                'e2': 40,
                'e3': 40,
                'e4': 30,
                'e5': 30,
                'e6': 20}

arc_lens_km = trans.get_arcs_length(arcs, node_coords)
arc_times_h = {k: v/arcs_avg_kmh[k] for k, v in arc_lens_km.items()}


od_pair = ('n1', 'n3')


# Component events
no_arc_st = 3 # number of component states 
delay_rat = [10, 2, 1] # delay in travel time given each component state (ratio)
vari = {}
for k, v in arcs.items():
    vari[k] = variable.Variable( name=k, B = np.eye( no_arc_st ), values = [arc_times_h[k]*np.float64(x) for x in delay_rat])


# plot graph
G = nx.Graph()
for k, x in arcs.items():
    G.add_edge(x[0], x[1], time=arc_times_h[k], label=k)

for k, v in node_coords.items():
    G.add_node(k, pos=v)

pos = nx.get_node_attributes(G, 'pos')
edge_labels = nx.get_edge_attributes(G, 'label')

fig = plt.figure()
ax = fig.add_subplot()
nx.draw(G, pos, with_labels=True, ax=ax)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
fig.savefig(os.path.join(HOME, 'graph.png'), dpi=200)

An example system function is as follows. <br>
Here, We define the system failure event as a travel time longer than thres*(normal time).

Two things to note about a system function: <br>
(1) It is preferred if a system function returns a possible scenario of components performance that has the highest probability of occurrence. <br>
(2) It is preferred if one can obtain a minimum required state of component vector from a result of system analysis. In case that this is not possible, the reference component vector for decomposition becomes the input state of component vector (which work okay by the proposed branch and bound algorithm).

In [2]:
def sys_fun( comps_st, od_pair, arcs, vari ):
    G = nx.Graph()
    for k, x in arcs.items():
        c_st = comps_st[k]
        G.add_edge(x[0], x[1], time=vari[k].values[c_st-1])

    path = nx.shortest_path( G, source = od_pair[0], target = od_pair[1], weight = 'time' )
    sys_val = nx.shortest_path_length( G, source = od_pair[0], target = od_pair[1], weight = 'time' )

    return sys_val, path

def state_from_sys_fun( sys_val, path, comps_st, thres, sys_val_itc, arcs ):
    if sys_val > thres*sys_val_itc:
        sys_st = 'fail'
    else:
        sys_st = 'surv'

    min_comps_st = {}
    for i in range(len(path)-1):
        nodes_i = [path[i], path[i+1]]
        nodes_i_rev = [path[i+1], path[i]] # reversed pair (arcs are bi-directional)
        arc_i = next((k for k, v in arcs.items() if v == nodes_i or v == nodes_i_rev), None)
        if arc_i is None:
            print(i)
        min_comps_st[arc_i] = comps_st[arc_i]

    return sys_st, min_comps_st


Given an intact state of component vector, we have a travel time as follows.

In [3]:
thres = 2 # defines the system failure event

# Intact state of component vector
comps_st_itc = {'e1': 3, 'e2': 3, 'e3': 3, 'e4': 3, 'e5': 3, 'e6': 3} # intact state

sys_val_itc, path_itc = sys_fun( comps_st_itc, od_pair, arcs, vari )
_, min_comps_itc = state_from_sys_fun( sys_val_itc, path_itc, comps_st_itc, thres, sys_val_itc, arcs )
print(sys_val_itc)
print(min_comps_itc)

0.18441968604480607
{'e2': 3, 'e5': 3}


Below are two example states of component vector.

In [4]:
# Example 1
comps_st1 = {'e1': 1, 'e2': 2, 'e3': 1, 'e4': 1, 'e5': 1, 'e6': 3}

sys_val1, path1 = sys_fun( comps_st1, od_pair, arcs, vari )
sys_st1, min_comps_st1 = state_from_sys_fun( sys_val1, path1, comps_st1, thres, sys_val_itc, arcs )

print(sys_val1)
print(path1)
print(sys_st1)
print(min_comps_st1)

# Example 2
comps_st2 = {'e1': 3, 'e2': 3, 'e3': 1, 'e4': 2, 'e5': 2, 'e6': 3}

sys_val2, path2 = sys_fun( comps_st2, od_pair, arcs, vari )
sys_st2, min_comps_st2 = state_from_sys_fun( sys_val2, path2, comps_st2, thres, sys_val_itc, arcs )

print(sys_val2)
print(path2)
print(sys_st2)
print(min_comps_st2)


1.1230866053552628
['n1', 'n5', 'n3']
fail
{'e2': 2, 'e5': 1}
0.2787005902030124
['n1', 'n5', 'n3']
surv
{'e2': 3, 'e5': 2}


## Branch and Bound algorithm

### Description

While we can start with any intial state, here we start with the intact state.

In [5]:
# Iteration 1
comps_st1 = {'e1': 3, 'e2': 2, 'e3': 3, 'e4': 3, 'e5': 3, 'e6': 3}

sys_val1, path1 = sys_fun( comps_st1, od_pair, arcs, vari )
sys_st1, min_comps_st1 = state_from_sys_fun( sys_val1, path1, comps_st1, thres, sys_val_itc, arcs )

print(sys_st1)
print(min_comps_st1)

surv
{'e2': 2, 'e5': 3}


In [6]:
# Iteration 2
comps_st2 = {'e1': 3, 'e2': 1, 'e3': 3, 'e4': 3, 'e5': 3, 'e6': 3}

sys_val2, path2 = sys_fun( comps_st2, od_pair, arcs, vari )
sys_st2, min_comps_st2 = state_from_sys_fun( sys_val2, path2, comps_st2, thres, sys_val_itc, arcs )

print(sys_st2)
print(min_comps_st2)

surv
{'e1': 3, 'e3': 3, 'e5': 3}


In [7]:
# Iteration 3
comps_st3 = {'e1': 3, 'e2': 3, 'e3': 3, 'e4': 3, 'e5': 2, 'e6': 3}

sys_val3, path3 = sys_fun( comps_st3, od_pair, arcs, vari )
sys_st3, min_comps_st3 = state_from_sys_fun( sys_val3, path3, comps_st3, thres, sys_val_itc, arcs )

print(sys_st3)
print(min_comps_st3)

surv
{'e2': 3, 'e6': 3, 'e4': 3}


In [8]:
# Iteration 4
comps_st4 = {'e1': 2, 'e2': 1, 'e3': 3, 'e4': 3, 'e5': 3, 'e6': 3}

sys_val4, path4 = sys_fun( comps_st4, od_pair, arcs, vari )
sys_st4, min_comps_st4 = state_from_sys_fun( sys_val4, path4, comps_st4, thres, sys_val_itc, arcs )

print(sys_st4)
print(min_comps_st4)

fail
{'e1': 2, 'e3': 3, 'e5': 3}


In [8]:
# Iteration 5
comps_st5 = {'e1': 3, 'e2': 1, 'e3': 2, 'e4': 3, 'e5': 3, 'e6': 3}

sys_val5, path5 = sys_fun( comps_st5, od_pair, arcs, vari )
sys_st5, min_comps_st5 = state_from_sys_fun( sys_val5, path5, comps_st5, thres, sys_val_itc, arcs )

print(sys_st5)
print(min_comps_st5)

fail
{'e1': 3, 'e3': 2, 'e5': 3}


### Now as an algorithm.

Function that selects the component and its state for next branch and bound. <br>
It selects (component, state) that is in a rule with the shortest length, and in case there are multiple rules with shortest length, the component that occurs the most across rules.

In [9]:
def get_comp_st_for_next_bnb( rule_s, rule_f ):
    
    rules = rule_s + rule_f
    r_len = [len(x) for x in rules]

    rl_min = min(r_len)
    cand_comps = [] # candidate components for next B&B -- These are the ones in rules with the shortest length
    for r in rules:
        if len(r) == rl_min:
            ks = [k for k in r.keys()]
            for k in ks:
                if k not in cand_comps:
                    cand_comps.append(k)


    cand_comps_cnt = {} # Count how many times the candidate components occur across all rules
    for x in cand_comps:
        in_r = [1 if x in r.keys() else 0 for r in rules]
        count_x = sum( np.array( in_r ) )
        cand_comps_cnt[x] = count_x


    comp_bnb = max(cand_comps_cnt, key=cand_comps_cnt.get)
    st_bnb = None
    for r in rule_s:
        if len(r) == rl_min:
            if comp_bnb in r.keys():
                st_bnb = r[comp_bnb]
                sys_bnb = 'surv'

                if rl_min == 1: # If there is only one condition left to satisfy
                    bnb_state = 'surv' # we know that the upper branch is always survive 
                else:
                    bnb_state = 'incomplete'

                break # Just get the first one occurring

    if st_bnb is None:
        for r in rule_f:
            if len(r) == rl_min:
                if comp_bnb in r.keys():
                    st_bnb = r[comp_bnb]
                    sys_bnb = 'fail'

                    if rl_min == 1: # If there is only one condition left to satisfy
                        bnb_state = 'fail' # we know that the lower branch is always survive 
                    else:
                        bnb_state = 'incomplete'

                    break # Just get the first one occurring

    return comp_bnb, st_bnb, sys_bnb, bnb_state

In [10]:
rule_s = [{'e2': 2, 'e5': 3}, {'e1': 3, 'e3': 3, 'e5': 3}, {'e2': 3, 'e4': 3, 'e6': 3}]
rule_f = [{'e1': 2, 'e2': 1}, {'e2': 1, 'e3': 2}]

comp_bnb, st_bnb, sys_bnb, bnb_state = get_comp_st_for_next_bnb( rule_s, rule_f )
print(comp_bnb)
print(st_bnb)
print(sys_bnb, bnb_state)

e2
2
surv incomplete


In [11]:
# function that converts a rule defined as a dictionary into a list
def rule_s_dict_to_list( r_dict, no_comp, comps_name_list, worst_st = 1 ):
    
    r_list = [worst_st] * no_comp

    for i in range( no_comp ):
        name_i = comps_name_list[i]

        if name_i in r_dict:
            r_list[i] = r_dict[name_i]

    return r_list


def rule_f_dict_to_list( r_dict, no_comp, comps_name_list, best_st ):
    
    r_list = [best_st] * no_comp

    for i in range( no_comp ):
        name_i = comps_name_list[i]

        if name_i in r_dict:
            r_list[i] = r_dict[name_i]

    return r_list


In [12]:
# Demonstration of functions above

no_comp = len(arcs)
comps_name = [k for k in arcs.keys()]

rule_s_dict = {'e2': 2, 'e5': 3}
rule_s_list = rule_s_dict_to_list( rule_s_dict, no_comp, comps_name, worst_st = 1 )
print(rule_s_list)

rule_f_dict = {'e1': 2, 'e2': 1}
rule_f_list = rule_f_dict_to_list( rule_f_dict, no_comp, comps_name, best_st= no_arc_st )
print( rule_f_list )

[1, 2, 1, 1, 3, 1]
[2, 1, 3, 3, 3, 3]


In [13]:
# function that converts list representation of a component vector state to dictionary
def comps_st_list_to_dict( st_list, comps_name_list ):
    
    st_dict = {comps_name_list[i]: st_list[i] for i in range( len(st_list) )}

    return st_dict

def comps_st_dict_to_list( st_dict, comps_name_list ):

    no_comp = len( comps_name_list )
    st_list = [None] * no_comp
    for i in range(no_comp):
        name_i = comps_name_list[i]
        st_list[ i ] = st_dict[name_i]

    return st_list


In [14]:
def get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_s_dict ):
    rs_cond = [] # remaining conditions to satisfy for compatible survival rules
    for r in rules_s_dict:
        if all( [ up_dict[k] >= r[k] for k in r] ): # the survival rule is compatible

            r_cond = {k:v for k,v in r.items() if down_dict[k] < r[k]}

            if len(r_cond) > 0:
                rs_cond.append( r_cond )
            
    return rs_cond


def get_rem_cond_for_compat_rule_f( down_dict, up_dict, rules_f_dict ):
    rf_cond = [] # remaining conditions to satisfy for compatible failure rules
    for r in rules_f_dict:
        if all( [ down_dict[k] <= r[k] for k in r] ): # the failure rule is compatible

            r_cond = {k:v for k,v in r.items() if up_dict[k] > r[k]}

            if len(r_cond) > 0:
                rf_cond.append( r_cond )
            
    return rf_cond


In [15]:
br1 = branch.Branch( [1,2,1,1,1,1], [3,3,3,3,3,3], is_complete=False )
rules_s = [{'e2': 2, 'e5': 3}, {'e1': 3, 'e3': 3, 'e5': 3}, {'e2': 3, 'e4': 3, 'e6': 3}]

down_dict = comps_st_list_to_dict( br1.down, comps_name )
up_dict = comps_st_list_to_dict( br1.up, comps_name )

rs_cond = get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_s )
print(rs_cond)


br1 = branch.Branch( [1,1,1,1,1,1], [3,1,3,3,3,3], is_complete=False )
rules_f = [{'e1': 2, 'e2': 1}, {'e2': 1, 'e3': 2}]

down_dict = comps_st_list_to_dict( br1.down, comps_name )
up_dict = comps_st_list_to_dict( br1.up, comps_name )

rf_cond = get_rem_cond_for_compat_rule_f( down_dict, up_dict, rules_f )
print(rf_cond)

[{'e5': 3}, {'e1': 3, 'e3': 3, 'e5': 3}, {'e2': 3, 'e4': 3, 'e6': 3}]
[{'e1': 2}, {'e3': 2}]


In [16]:
rules_s = [min_comps_itc]
rule_s_ind_cpt = []
for i in range( len(rules_s) ):
    r_i = rules_s[i]

    r_i_list = []

In [17]:
comps_name_list = [k for k in arcs.keys()]

def decomp_to_two_branches( br1, comp_bnb, st_bnb, sys_st_bnb, comps_name_list ):

    down_dict = comps_st_list_to_dict( br1.down, comps_name )
    up_dict = comps_st_list_to_dict( br1.up, comps_name )

    if sys_st_bnb == 'surv':
        up_dict_bl = copy.deepcopy(up_dict) # the branch on the lower side
        up_dict_bl[comp_bnb] = st_bnb - 1

        down_dict_bu = copy.deepcopy(down_dict) # the branch on the upper side
        down_dict_bu[comp_bnb] = st_bnb

    else: # i.e. sys_st_bnb == 'fail'
        up_dict_bl = copy.deepcopy(up_dict)
        up_dict_bl[comp_bnb] = st_bnb

        down_dict_bu = copy.deepcopy(down_dict)
        down_dict_bu[comp_bnb] = st_bnb + 1
        

    up_list_bl = comps_st_dict_to_list( up_dict_bl, comps_name_list )
    down_list_bu = comps_st_dict_to_list(down_dict_bu, comps_name_list)
    new_brs_list = [branch.Branch( br1.down, up_list_bl, is_complete=False ),
                    branch.Branch( down_list_bu, br1.up, is_complete=False )]
    
    return new_brs_list


# Example 1
comp_bnb = 'e2'
st_bnb = 3
sys_st_bnb = 'surv'

br1 = branch.Branch( [1] * len(arcs), [no_arc_st] * len(arcs), is_complete=False )
new_brs = decomp_to_two_branches( br1, comp_bnb, st_bnb, sys_st_bnb, comps_name_list )
print(new_brs)


# Exmple 2
comp_bnb = 'e2'
st_bnb = 1
sys_st_bnb = 'fail'

br1 = branch.Branch( [1] * len(arcs), [no_arc_st] * len(arcs), is_complete=False )
new_brs = decomp_to_two_branches( br1, comp_bnb, st_bnb, sys_st_bnb, comps_name_list )
print(new_brs)

[Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 2, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None, Branch(down=[1, 3, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None]
[Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 1, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None, Branch(down=[1, 2, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None]


#### Func: Update a rules list by removing dominated rules and adding a new rule

In [18]:
def add_a_new_rule( rules_list, rule1, fail_or_surv ):

    r_rmv_inds = []
    for i in range(len(rules_list)):
        r_i = rules_list[i]

        if all( [True if k in r_i else False for k in rule1.keys()] ): # does all keys in rule 1 exist for rule_i?

            if fail_or_surv == 'surv':            
                if all( [True if r_i[k] >= v else False for k,v in rule1.items()] ): # this rule is dominated by the new rule
                    r_rmv_inds += [i]           
                
            else: # fail_or_surv == 'fail'            
                if all( [True if r_i[k] <= v else False for k,v in rule1.items()] ): # this rule is dominated by the new rule
                    r_rmv_inds += [i]

    rules_list_new = copy.deepcopy( rules_list )
    for i in r_rmv_inds:
        del rules_list_new[i]

    rules_list_new += [rule1]

    return rules_list_new


# Example 1: survival case
rules_s = [{'e2': 3, 'e5': 3}] # a list of rules
rs_new = {'e2': 2, 'e5': 3} # a new rule

rules_s_new = add_a_new_rule( rules_s, rs_new, 'surv' )
print(rules_s_new)

# Exmaple 2: failure case
rules_f = [{'e2': 2, 'e5': 2}] # a list of rules
rf_new = {'e2': 2, 'e5': 1} # a new rule

rules_f_new = add_a_new_rule( rules_f, rf_new, 'surv' )
print(rules_f_new)



[{'e2': 2, 'e5': 3}]
[{'e2': 2, 'e5': 1}]


## Branch and bound by algorithm

### System function

A system function needs to return (1) system function value, (2) system state, and (3) minimally required component state to fulfill the obtained system function value. If (3) is unavailable, it can be returned as None. <br>
It requires input being a component state in regard to which a system analysis needs to be done.

In [26]:
# Example 1: when minimally required component state is available.
def sf_min_path( comps_st, od_pair, arcs, vari, thres, sys_val_itc ):

    G = nx.Graph()
    for k, x in arcs.items():
        c_st = comps_st[k]
        G.add_edge(x[0], x[1], time=vari[k].values[c_st-1])

    path = nx.shortest_path( G, source = od_pair[0], target = od_pair[1], weight = 'time' )
    sys_val = nx.shortest_path_length( G, source = od_pair[0], target = od_pair[1], weight = 'time' )

    if sys_val > thres*sys_val_itc:
        sys_st = 'fail'
    else:
        sys_st = 'surv'

    if sys_st == 'surv': # in this case we know how to find out minimally required component state
        min_comps_st = {}
        for i in range(len(path)-1):
            nodes_i = [path[i], path[i+1]]
            nodes_i_rev = [path[i+1], path[i]] # reversed pair (arcs are bi-directional)
            arc_i = next((k for k, v in arcs.items() if v == nodes_i or v == nodes_i_rev), None)
            min_comps_st[arc_i] = comps_st[arc_i]
    
    else: # sys_st == 'fail'
        min_comps_st = None

    return sys_val, sys_st, min_comps_st


# Given a system function, i.e. sf_min_path, it should be represented by a function that only has "comps_st" as input.
sys_fun = lambda comps_st : sf_min_path( comps_st, od_pair, arcs, vari, thres, sys_val_itc ) # TODO: branch needs to have states defined in dictionary instead of list.

sys_val, sys_st, min_comps_st = sys_fun( {'e1': 3, 'e2': 1, 'e3': 2, 'e4': 3, 'e5': 3, 'e6': 3} )
print(sys_val, sys_st, min_comps_st)

0.4245584679314058 fail None


### Initialisation

In [37]:
#initialisation
max_br = 1000 # user input: maximum no. of branches 
no_sf = 0 # number of system function runs so far
sys_res = pd.DataFrame(data={'sys_val': [], 'comps_st': [], 'comps_st_min': []}) # system function results 

rules_s = [] # a list of known survival rules
rules_f = [] # a list of known failure rules



In [38]:
no_iter =  0
while no_iter < 20: # for debugging 

    ### for debugging ###
    no_iter += 1 
    print( 'Iteration: ', no_iter ) 

    print( 'Known rule--surv: ' )
    for d in rules_s:
        print( d )
    print( 'Known rule--fail: ' )
    for d in rules_f:
        print( d )
    #####################

    ## Start from the total event
    down_list = [1] * len(arcs) # all components in the worst state
    up_list = [len( vari[e].B[0] ) for e in comps_name] # all components in the best state

    brs = [branch.Branch( down_list, up_list, is_complete=False )]
    brs_rs_cond = [rules_s] # compatible survival rules with only yet satisfied conditions stored (have the same length as brs)
    brs_rf_cond = [rules_f] # compatible failure rules with only yet satisfied conditions stored (have the same length as brs)

    no_br_stage = 0 # for debugging
    while sum( [1 if b.is_complete == False else 0 for b in brs ] ) > 0:

        ### for debugging ###
        no_br_stage += 1 
        print( 'Branching stage: ', no_br_stage ) 
        #####################
        
        brs_new = []
        brs_rs_cond_new = []
        brs_rf_cond_new = []
        for i in range(len( brs )):

            br_i = brs[i]
            rs_cond_i = brs_rs_cond[i]
            rf_cond_i = brs_rf_cond[i]

            ### for debugging ###
            print( 'Branch #: ', i+1 ) 
            print( 'Branch: ', br_i )

            print( 'Compatible rules--surv:' )
            for d in rs_cond_i:
                print( d )
            print( 'Compatible rules--fail: ' )
            for d in rf_cond_i:
                print( d )
            #####################

            if len(rs_cond_i) == 0 and len(rf_cond_i) == 0 and br_i.is_complete == False:

                brs_new += [br_i]
                brs_rs_cond += [[]]
                brs_rf_cond += [[]]
                break # exit and run system function on this branch's upper bound

            elif br_i.is_complete ==  True: # no need to do branch if the branch's state is known
                brs_new += [br_i] 
                brs_rs_cond += [[]]
                brs_rf_cond += [[]]
            
            elif br_i.is_complete == False: # branch this branch 
                comp_bnb_i, st_bnb_i, sys_bnb_i, bnb_st_i = get_comp_st_for_next_bnb( rs_cond_i, rf_cond_i )
                brs_new_i = decomp_to_two_branches( br_i, comp_bnb_i, st_bnb_i, sys_bnb_i, comps_name_list ) # comps_name_list is not necessary if branch's down and up are defined in dictionary
                
                if bnb_st_i == 'surv': # we know that the upper branch is a survival one
                    brs_new_i[1].down_state = 'surv'
                    brs_new_i[1].up_state = 'surv'
                    brs_new_i[1].is_complete = True

                elif bnb_st_i == 'fail': # we know that the lower branch is a failure one
                    brs_new_i[0].down_state = 'fail'
                    brs_new_i[0].up_state = 'fail'
                    brs_new_i[0].is_complete = True
                
                brs_new += brs_new_i

                for b in brs_new_i: # update compatible rules and their remaining conditions to satisfy
                    
                    down_dict_b = comps_st_list_to_dict( b.down, comps_name_list )
                    up_dict_b = comps_st_list_to_dict( b.up, comps_name_list )

                    rs_cond_b = get_rem_cond_for_compat_rule_s( down_dict_b, up_dict_b, rs_cond_i )
                    rf_cond_b = get_rem_cond_for_compat_rule_f( down_dict_b, up_dict_b, rf_cond_i )

                    brs_rs_cond_new += [rs_cond_b]
                    brs_rf_cond_new += [rf_cond_b]


                ### for debugging ###
                print( 'New branch--low: ', brs_new_i[0] ) 
                print( '   compat. rules--surv: ', brs_rs_cond_new[-2] )
                print( '   compat. rules--fail: ', brs_rf_cond_new[-2] )

                print( 'New branch--up: ', brs_new_i[1] )
                print( '   compat. rules--surv: ', brs_rs_cond_new[-1] )
                print( '   compat. rules--fail: ', brs_rf_cond_new[-1] )
                #####################


                if len(brs_new) > max_br:
                    break # stop the process if there are too many branches

        brs = copy.deepcopy( brs_new )
        brs_rs_cond = copy.deepcopy( brs_rs_cond_new )
        brs_rf_cond = copy.deepcopy( brs_rf_cond_new )

        if len(brs_new) > max_br:
            break 

    # run system function
    up_dict_i = comps_st_list_to_dict( br_i.up, comps_name )


    no_sf += 1
    sys_val_i, sys_st_i, min_comps_st_i = sys_fun( up_dict_i )

    sys_res = pd.concat( [sys_res,
                        pd.DataFrame( {'sys_val': [sys_val_i], 'comps_st': [up_dict_i], 'comps_st_min': [min_comps_st_i]} )],
                        ignore_index = True )


    if sys_st_i == 'surv':
        if min_comps_st_i is not None:
            rs_new = min_comps_st_i
        else:
            rs_new = { k:v for k,v in up_dict_i.items() if v > 1 } # the rule is the same as up_dict_i but includes only components whose state is greater than the worst one (i.e. 1)
        
        rules_s = add_a_new_rule( rules_s, rs_new, 'surv' )


    else: # sys_st_i == 'fail'
        if min_comps_st_i is not None:
            rf_new = min_comps_st_i
        else:
            rf_new = { k:v for k,v in up_dict_i.items() if v < len(vari[k].B[0]) } # the rule is the same as up_dict_i but includes only components whose state is less than the best one

        rules_f = add_a_new_rule( rules_f, rf_new, 'fail' )


    ### for debugging ###
    print( 'Survival rules: ', rules_s ) 
    print( 'Failure rules: ', rules_f ) 
    #####################


#### 151023: seems to work but something is wrong

Iteration:  1
Known rule--surv: 
Known rule--fail: 
Branching stage:  1
Branch #:  1
Branch:  Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None
Compatible rules--surv:
Compatible rules--fail: 
Branching stage:  2


IndexError: list index out of range

In [32]:
print( brs_new )

[]


In [177]:
down_list = [1] * len(arcs) # all components in the worst state
up_list = [len( vari[e].B[0] ) for e in comps_name] # all components in the best state

brs = [branch.Branch( down_list, up_list, is_complete=False )]
brs_rs_cond = [rules_s] # compatible survival rules with only yet satisfied conditions stored (have the same length as brs)
brs_rf_cond = [rules_f] # compatible failure rules with only yet satisfied conditions stored (have the same length as brs)

In [179]:
   
brs_new = []
brs_rs_cond_new = []
brs_rf_cond_new = []
for i in range(len( brs )):

    br_i = brs[i]
    rs_cond_i = brs_rs_cond[i]
    rf_cond_i = brs_rf_cond[i]

    if len(rs_cond_i) == 0 and len(rf_cond_i) == 0 and br_i.is_complete == False:
        break # exit and run system function on this branch's upper bound
    
    elif br_i.is_complete == False: # branch this branch (no need to do branch if the branch's state is known)
        comp_bnb_i, st_bnb_i, sys_bnb_i, bnb_st_i = get_comp_st_for_next_bnb( rs_cond_i, rf_cond_i )
        brs_new_i = decomp_to_two_branches( br_i, comp_bnb_i, st_bnb_i, sys_bnb_i, comps_name_list ) # comps_name_list is not necessary if branch's down and up are defined in dictionary
        
        if bnb_st_i == 'surv': # we know that the upper branch is a survival one
            brs_new_i[1].down_state = 'surv'
            brs_new_i[1].up_state = 'surv'
            brs_new_i[1].is_complete = True

        elif bnb_st_i == 'fail': # we know that the lower branch is a failure one
            brs_new_i[0].down_state = 'fail'
            brs_new_i[0].up_state = 'fail'
            brs_new_i[0].is_complete = True
        
        brs_new += brs_new_i

        for b in brs_new_i: # update compatible rules and their remaining conditions to satisfy
            
            down_dict_b = comps_st_list_to_dict( b.down, comps_name_list )
            up_dict_b = comps_st_list_to_dict( b.up, comps_name_list )

            rs_cond_b = get_rem_cond_for_compat_rule_s( down_dict_b, up_dict_b, rs_cond_i )
            rf_cond_b = get_rem_cond_for_compat_rule_f( down_dict_b, up_dict_b, rf_cond_i )

            brs_rs_cond_new += [rs_cond_b]
            brs_rf_cond_new += [rf_cond_b]

brs

print(brs_new)

[Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 1, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None, Branch(down=[1, 2, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None]


In [ ]:
rs_cond_i = brs_rs_cond[i]
rf_cond_i = brs_rf_cond[i]

comp_b_i, st_b_i, sys_b_i = get_comp_st_for_next_bnb(rs_cond_i, rf_cond_i) 

brs_new_i = decomp_to_two_branches( br_i, comp_b_i, st_b_i, sys_b_i, comps_name_list )
brs_new += brs_new_i


#### 131023: Continue from getting rule conditions for the new branches ####
rs_cond_i = get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_s )
rf_cond_i = get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_f )

"""
if len(rs_cond_i) < 1 and len(rf_cond_i) < 1: # no compatible rules left--stop and use this branch for next system analysis
    break
else:
"""

In [ ]:






worst_st = [1] * len(arcs)



rules_s = [min_comps_itc]
rules_f = []
print(rules_s)


brs = [branch.Branch( worst_st, best_st, is_complete=False )]

down_dict = comps_st_list_to_dict( brs[0].down, comps_name )
up_dict = comps_st_list_to_dict( brs[0].up, comps_name )


brs_rs_cond = [ get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_s ) ]
brs_rf_cond = [[]]


In [22]:






## Start with the total event ##
brs = [branch.Branch( worst_st, best_st, is_complete=False )]

down_dict = comps_st_list_to_dict( brs[0].down, comps_name )
up_dict = comps_st_list_to_dict( brs[0].up, comps_name )

brs_rs_cond = [ get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_s ) ]
brs_rf_cond = [[]]
#################################


# while (there are any incomplete branches) or (the number of branches exceed what user defined <-- actually this should be inside the while loop)
brs_new = [] # the length becomes twice as the original brs if branches are completed
for i in range( len(brs) ):

    br_i = brs[i]
    
    down_dict = comps_st_list_to_dict( br_i.down, comps_name )
    up_dict = comps_st_list_to_dict( br_i.up, comps_name )

    rs_cond_i = brs_rs_cond[i]
    rf_cond_i = brs_rf_cond[i]

    comp_b_i, st_b_i, sys_b_i = get_comp_st_for_next_bnb(rs_cond_i, rf_cond_i) 
    
    brs_new_i = decomp_to_two_branches( br_i, comp_b_i, st_b_i, sys_b_i, comps_name_list )
    brs_new += brs_new_i


    #### 131023: Continue from getting rule conditions for the new branches ####
    rs_cond_i = get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_s )
    rf_cond_i = get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_f )

    """
    if len(rs_cond_i) < 1 and len(rf_cond_i) < 1: # no compatible rules left--stop and use this branch for next system analysis
        break
    else:
    """
    
        

    brs_rs_cond[i] = rs_cond_i
    brs_rf_cond[i] = rf_cond_i


print(brs_rs_cond)
print(brs_rf_cond)

# pick up a branch as soon as 

[{'e2': 3, 'e5': 3}]
e2 3 surv
[[{'e2': 3, 'e5': 3}]]
[[]]


In [60]:




no_incomp_br = sum( 1 for x in brs if x.is_complete == False ) # number of incomplete branches
#while len(brs) < max_br and no_incomp_br > 0:

comp_bnb, st_bnb, sys_bnb = get_comp_st_for_next_bnb( rules_s, rules_f )
print(comp_bnb)
print(st_bnb)

brs = [branch.Branch( worst_st, best_st, is_complete=False )] # total event
#for b in brs:
b = brs[0]
print(b)

#if b.is_complete == False:
down_dict = comps_st_list_to_dict( b.down, comps_name )
up_dict = comps_st_list_to_dict( b.up, comps_name )

rs_cond = get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_s )
rf_cond = get_rem_cond_for_compat_rule_s( down_dict, up_dict, rules_f )
print(rs_cond)
print(rf_cond)

comp_bnb, st_bnb, sys_bnb = get_comp_st_for_next_bnb( rule_s, rule_f )
print(comp_bnb)
print(st_bnb)
print(sys_bnb)





[{'e2': 3, 'e5': 3}]
e2
3
Branch(down=[1, 1, 1, 1, 1, 1], up=[3, 3, 3, 3, 3, 3], is_complete=False, down_state=1, up_state=1, down_val=None, up_val=None
[{'e2': 3, 'e5': 3}]
[]
e2
2
surv


In [111]:
print(no_incomp_br)

1


In [ ]:
brs = []
print(isemtpy)